In [17]:
import cv2

# Load the pre-trained Haar cascade for eye detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Function to detect eyes in an image and return their positions
def detect_eyes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
    print(f"This is eye{eyes}")
    return eyes

# Function to estimate the gaze direction based on the eye positions
def estimate_gaze(eyes, image):
    height, width, _ = image.shape
    eye_centers = []
    for (x, y, w, h) in eyes:
        center_x = x + w // 2
        center_y = y + h // 2
        eye_centers.append((center_x, center_y))

        # Draw bounding boxes around the eyes
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(image, (center_x, center_y), 2, (0, 0, 255), 2)
        
    # Estimate the gaze direction based on the eye positions
    # if len(eye_centers) == 2:
    #     eye1_x, eye1_y = eye_centers[0]
    #     eye2_x, eye2_y = eye_centers[1]

    #     # Calculate the average of eye coordinates as the point of gaze
    #     gaze_x = (eye1_x + eye2_x) // 2
    #     gaze_y = (eye1_y + eye2_y) // 2

    #     print(f"User is looking at coordinates: ({gaze_x}, {gaze_y})")

    #     # Determine the gaze direction based on the differences in x and y coordinates
    #     if gaze_y < height // 3:
    #         # Look up
    #         print("User is looking up")
    #         cv2.circle(image, (gaze_x, gaze_y), 10, (0, 0, 255), -1)
    #     elif gaze_y > (2 * height) // 3:
    #         # Look down
    #         print("User is looking down")
    #         cv2.circle(image, (gaze_x, gaze_y), 10, (0, 0, 255), -1)
    #     elif gaze_x < width // 3:
    #         # Look left
    #         print("User is looking left")
    #         cv2.circle(image, (gaze_x, gaze_y), 10, (0, 0, 255), -1)
    #     elif gaze_x > (2 * width) // 3:
    #         # Look right
    #         print("User is looking right")
    #         cv2.circle(image, (gaze_x, gaze_y), 10, (0, 0, 255), -1)
    #     else:
    #         # Look straight ahead
    #         print("User is looking straight ahead")
    #         cv2.circle(image, (gaze_x, gaze_y), 10, (0, 0, 255), -1)

    return image

# Main loop to capture video from the webcam and detect eyes
def main():
    # Open the webcam
    cap = cv2.VideoCapture(0)

    while True:
        # Read a frame from the webcam
        ret, frame = cap.read()

        # Flip the frame horizontally for a mirror effect
        frame = cv2.flip(frame, 1)

        # Detect eyes in the frame
        eyes = detect_eyes(frame)

        # Estimate gaze direction and draw visualizations
        output_frame = estimate_gaze(eyes, frame)

        # Display the resulting frame
        cv2.imshow('Eye Detection', output_frame)

        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close the OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


This is eye[[335 305  61  61]
 [228 287  88  88]]
This is eye[[227 287  89  89]]
This is eye[[332 303  64  64]
 [224 286  90  90]]
This is eye[[336 304  61  61]
 [229 286  87  87]]
This is eye[[334 303  64  64]
 [229 283  91  91]]
This is eye[[332 305  56  56]
 [232 285  89  89]]
This is eye[[336 302  63  63]
 [229 287  86  86]]
This is eye[[334 302  62  62]
 [228 283  92  92]]
This is eye[[332 299  63  63]
 [232 281  93  93]]
This is eye[[336 303  67  67]
 [232 286  89  89]]
This is eye[[335 296  69  69]
 [233 284  92  92]]
This is eye[[337 300  65  65]
 [232 285  88  88]]
This is eye[[333 296  69  69]
 [235 284  90  90]]
This is eye[[333 298  68  68]
 [232 285  88  88]]
This is eye[[336 299  67  67]
 [230 287  84  84]]
This is eye[[344 306  59  59]
 [231 285  86  86]]
This is eye[[233 285  88  88]
 [339 301  64  64]]
This is eye[[342 302  63  63]
 [233 286  84  84]]
This is eye[[232 287  85  85]
 [328 286  86  86]]
This is eye[[342 302  64  64]
 [229 285  88  88]]
This is eye[[338 29

In [3]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
model = load_model('eye_model.h5')
folders = ['bottom_left', 'bottom_right', 'bottom_middle', 'top_left', 'top_right', 'top_middle', 'left_middle', 'right_middle']



# Load your pre-trained CNN model

# Define the desired dimensions
desired_width = 250
desired_height = 50

# Initialize the webcam
video_capture = cv2.VideoCapture(0)  # 0 represents the default webcam device

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = video_capture.read()

    # Flip the frame horizontally
    # frame = cv2.flip(frame, 1)

    # Define the ROI coordinates for the eye portion
    eye_x = 100
    eye_y = 100
    eye_width = 150
    eye_height = 50

    # Extract the eye ROI from the frame
    eye_roi = frame[eye_y:eye_y+eye_height, eye_x:eye_x+eye_width]

    # Resize the ROI to match the desired dimensions
    resized_roi = cv2.resize(eye_roi, (desired_width, desired_height))

    # Convert the ROI to RGB format
    resized_roi = cv2.cvtColor(resized_roi, cv2.COLOR_BGR2RGB)

    # Convert the ROI to numpy array
    resized_roi = np.asarray(resized_roi)

    # Expand dimensions to match the input shape of your CNN model
    resized_roi = np.expand_dims(resized_roi, axis=0)

    # Normalize pixel values
    resized_roi = resized_roi / 255.0

    # Pass the preprocessed ROI through your CNN model for prediction
    predictions = model.predict(resized_roi)
    predicted_class = np.argmax(predictions, axis=1)

    # Display the predicted class
    print(folders[predicted_class[0]])
    # Display the frame with overlays or visual indicators of person detection
    cv2.imshow('Eye Detection', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
video_capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 89ms/step
bottom_middle
1/1 [==============================] - 0s 63ms/step
bottom_middle
1/1 [==============================] - 0s 50ms/step
bottom_middle
1/1 [==============================] - 0s 34ms/step
bottom_middle
1/1 [==============================] - 0s 35ms/step
bottom_middle
1/1 [==============================] - 0s 31ms/step
bottom_middle
1/1 [==============================] - 0s 34ms/step
bottom_middle
1/1 [==============================] - 0s 29ms/step
bottom_middle
1/1 [==============================] - 0s 33ms/step
bottom_middle
1/1 [==============================] - 0s 37ms/step
bottom_middle
1/1 [==============================] - 0s 32ms/step
bottom_middle
1/1 [==============================] - 0s 31ms/step
bottom_middle
1/1 [==============================] - 0s 36ms/step
bottom_middle
1/1 [==============================] - 0s 35ms/step
bottom_middle
1/1 [==============================] - 0s 30ms/step
bottom_middle
1/1 [=====

In [4]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import time

model = load_model('eye_model_by_saurabey.h5')
folders = ['bottom_left', 'bottom_right', 'bottom_middle', 'top_left', 'top_right', 'top_middle', 'left_middle', 'right_middle']

eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load your pre-trained CNN model
def detect_eyes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
    return eyes


# Define the desired dimensions
desired_width = 250
desired_height = 50

# Create the folder if it doesn't exist
folder_path = 'eyes_pattern'
os.makedirs(folder_path, exist_ok=True)

# Initialize the webcam
video_capture = cv2.VideoCapture(0)  # 0 represents the default webcam device

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = video_capture.read()
    eyes = detect_eyes(frame)

    if len(eyes) >= 2:
        (eye_1x, eye_1y, eye_1width, eye_1height), (eye_2x, eye_2y, eye_2width, eye_2height) = eyes[:2]
        eye_x = min(eye_1x, eye_2x)
        eye_y = min(eye_1y, eye_2y)
        eye_width = abs(eye_1x - eye_2x) + max(eye_1width, eye_2width)
        eye_height = abs(eye_1y - eye_2y) + max(eye_1height, eye_2height)
        
        eye_roi = frame[eye_y:eye_y+eye_height, eye_x:eye_x+eye_width]

        # Resize the ROI to match the desired dimensions
        resized_roi = cv2.resize(eye_roi, (desired_width, desired_height))

        # Convert the ROI to RGB format
        resized_roi = cv2.cvtColor(resized_roi, cv2.COLOR_BGR2RGB)

        # Convert the ROI to numpy array
        resized_roi = np.asarray(resized_roi)

        # Expand dimensions to match the input shape of your CNN model
        resized_roi = np.expand_dims(resized_roi, axis=0)

        # Normalize pixel values
        resized_roi = resized_roi / 255.0

        # Pass the preprocessed ROI through your CNN model for prediction
        predictions = model.predict(resized_roi)
        predicted_class = np.argmax(predictions, axis=1)
        # print(predicted_class)
        frame = cv2.rectangle(frame, (eye_x, eye_y), (eye_x+eye_width, eye_y+eye_height), (0, 255, 0), 2)
        frame = cv2.putText(frame, folders[predicted_class[0]], (eye_x, eye_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        # if predicted_class == 0:
        #     print("bottom left corner")
        # if predicted_class == 1:
        #     print("bottom right corner")
        # # if predicted_class == 2:
        # #     print("bottom middle")
        # # if predicted_class == 3:
        # #     print("top left corner")
        # # if predicted_class == 4:
        # #     print("top right corner")
        # # if predicted_class == 5:
        # #     print("top middle")
        # # if predicted_class == 6:
        # #     print("left middle")
        # # if predicted_class == 7:
        # #     print("right middle")

        

    # Display the frame with overlays or visual indicators of eye detection
    frame = cv2.flip(frame, 1)
    cv2.imshow('Eye Detection', frame)

    # Add a small delay to reduce processing load
    # time.sleep(0.01)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
video_capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 31ms/step


IndexError: list index out of range

: 

In [ ]:

# Save the frame in the "eyes_pattern" folder
frame_count += 1
filename = os.path.join(folder_path, f'frame_{frame_count}.jpg')
cv2.imwrite(filename, eye_roi)